# Sunmat tutorial

*Author: Audun Skau Hansen (a.s.hansen@kjemi.uio.no) , January 2019*

This document gives a brief description of the sunmat module. As an example application, we use a gradient to find an arbitrary rotation of a 5 dimensional unit vector. You'll need the module <a href="https://github.com/HIPS/autograd">autograd</a> to run this notebook.

### Quick theoretical background

---

This module facilitates the generation of parametrized matrices with matrix elements $U_{ij} \in \mathbb{R}$

\begin{equation}
U(\varphi_1, \varphi_2, ...),
\end{equation}

with the property

\begin{equation}
U^T(\varphi_1, \varphi_2, ...) U(\varphi_1, \varphi_2, ...) = \mathbb{1}
\end{equation}


<a href="https://en.wikipedia.org/wiki/Special_unitary_group">More information on SU(n) representations</a>

### Example

For a random matrix A, with uniformly distributed elements $A_{ij} \in (-1,1)$, we may construct a random anti-hermitean matrix $H$ as

\begin{equation}
H = \exp\left( A^T - A \right)  = \sum_{n=0}^\infty \frac{\left(A^T - A\right)^n }{n!},
\end{equation}

where $\left( A^T - A \right)^0 = \mathbb{1}$.

We may thus generate a random such matrix as follows:


In [1]:
from scipy.linalg import expm
import autograd.numpy as np # 
import autograd as ag

def randU(n, m):
    A = np.random.uniform(-m,m, (n,n))
    return expm(A.T - A)

Confirm that this matrix is within the space of the su(n) matrices:

In [2]:
H = randU(4, 1)
print("H=")
print(H)
print(" ")
print("H^T H =")
print(np.dot(H.T, H)) #Should be the 4 by 4 identity matrix to numerical precision

H=
[[ 0.54324137 -0.65767956 -0.22628149  0.47025854]
 [ 0.8212089   0.44990517  0.3058028  -0.17229607]
 [-0.16461522 -0.13478668  0.87982667  0.42501695]
 [-0.058366    0.58896136 -0.2849322   0.75401032]]
 
H^T H =
[[ 1.00000000e+00  8.99391245e-17  1.33176223e-17  3.90638630e-17]
 [ 8.99391245e-17  1.00000000e+00  4.28563801e-17  6.85175006e-18]
 [ 1.33176223e-17  4.28563801e-17  1.00000000e+00 -4.78418310e-17]
 [ 3.90638630e-17  6.85175006e-18 -4.78418310e-17  1.00000000e+00]]


Next, we want to reconstruct this matrix using the parametrized sunmat:

In [3]:
import sunmat as sm

In [4]:

n,u = sm.full_suN(4) #generate su(4) matrix
#n,u = sm.suNia(4, [0,1,2,3], [0,1,2,3]) #optional more control of which rotations to include

print("number of parameters:", n)
print("Identity:")
print(u(np.zeros(n, dtype = float)))

number of parameters: 6
Identity:
[[ 1. -0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1. -0.]
 [ 0.  0.  0.  1.]]


We need an objective function to measure how closely the u matrix resembles H, so we define a matrix norm (omitting the square root):

In [5]:
def mat_diff(x):
    return np.sum((u(x) - H)**2)


In [6]:
print("Deviation from unity:", mat_diff(np.zeros(n, dtype = float)))

Deviation from unity: 2.746032931840328


To perform a crude optimization of this problem we need the gradient, so we use autograd:

In [7]:
dm = ag.grad(mat_diff)

Starting out from the identity matrix, we then simply follow the gradient naïvely:

In [10]:
x_min = np.zeros(n, dtype = float)
for i in np.arange(100):
    x_min -= .1*dm(x_min)
    
print("Final objective value:", mat_diff(x_min))
print("Parameters:")
print(x_min)
print("Final u(x_min) = ")
print(u(x_min))
print("H= ")
print(H)

Final objective value: 9.076527510687277e-18
Parameters:
[ 0.98636928 -0.1656529  -0.05839919 -0.16283925  0.6310181  -0.36130107]
Final u(x_min) = 
[[ 0.54324137 -0.65767956 -0.22628149  0.47025854]
 [ 0.8212089   0.44990517  0.3058028  -0.17229607]
 [-0.16461522 -0.13478668  0.87982667  0.42501694]
 [-0.058366    0.58896136 -0.2849322   0.75401033]]
H= 
[[ 0.54324137 -0.65767956 -0.22628149  0.47025854]
 [ 0.8212089   0.44990517  0.3058028  -0.17229607]
 [-0.16461522 -0.13478668  0.87982667  0.42501695]
 [-0.058366    0.58896136 -0.2849322   0.75401032]]


In summary, we should see that the parameters converge toward reasonable values.